In [ ]:
##################################
#We run some analysis on the data we have processes
##################################

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

In [ ]:
#valuation dates
valuation_dates = ['2019-12-31','2018-12-31','2017-12-31','2016-12-31','2015-12-31','2014-12-31','2013-12-31','2012-12-31','2011-12-31','2010-12-31','2009-12-31','2008-12-31','2007-12-31']
#paths to saved data
data_path ='./financials.txt'
model_data_path ='./model_input.txt'
#we will revalue everything to USD at FX 31/12/2019 rates of exchange
ccy_fx = {'AUD' : 0.70209,
          'CAD': 0.7686,
          'CHF': 1.0339,
          'EUR': 1.122,
          'GBP': 1.326,
          'HKD': 0.12835,
          'ILS': 0.2896,
          'JPY':0.0092,
          'RUB':0.0161,
          'SGD':0.7493,
          'USD':1,
         }

In [ ]:
# we prepare the data into our training set
financials_df= pd.read_csv(data_path,sep='\t',low_memory=False)
financials_df.drop(columns='Unnamed: 0',inplace=True)
financials_df = financials_df.loc[financials_df[valuation_dates[-1]] >0].reset_index(drop=True)

#remove entries without numbers of shares
#common shares outstanding
financials_df['QTCO'] = financials_df['QTCO'].fillna(0)
financials_df = financials_df.loc[financials_df['QTCO'] >0].reset_index(drop=True)

#remove entries with no final valuation amounts
financials_df = financials_df.loc[financials_df['2019-12-31'] >0].reset_index(drop=True)

#remove entries with no valuation amounts at the end of 2016
financials_df = financials_df.loc[financials_df['2016-12-31'] >0].reset_index(drop=True)
financials_df['Source_Year']=financials_df['SourceDate'].str[0:4].astype(int)
financials_df['Source_Month']=financials_df['SourceDate'].str[5:7].astype(int)
financials_df['Source_Day']=financials_df['SourceDate'].str[-2:].astype(int)
financials_df['Period_Year']=financials_df['PeriodEndDate'].str[0:4].astype(int)
financials_df['Period_Month']=financials_df['PeriodEndDate'].str[5:7].astype(int)
financials_df['Period_Day']=financials_df['PeriodEndDate'].str[-2:].astype(int)

financials_df.drop(columns=['PeriodEndDate','SourceDate'],inplace=True)

#add in the FX rate of exchange
financials_df['FX_USD']=financials_df['Currency'].apply(lambda x:ccy_fx[x])

#update the valuation dates to hold the USD market Caps
for val_date in valuation_dates:
    financials_df[val_date]=financials_df['FX_USD']*financials_df['QTCO']*financials_df[val_date]

year_stock_ccy_df = financials_df[['Source_Year','Symbol','Country']]

with pd.option_context('display.max_rows', 50, 'display.max_columns', None):
    display(pd.pivot_table(year_stock_ccy_df, index=['Country'], columns=['Source_Year'], values=['Symbol'], aggfunc=np.count_nonzero, margins=True).fillna(0))

year_stock_ccy_df = financials_df[['Source_Year','Symbol','Currency']].drop_duplicates()

with pd.option_context('display.max_rows', 50, 'display.max_columns', None):
    display(pd.pivot_table(year_stock_ccy_df, index=['Source_Year'], columns=['Currency'], values=['Symbol'], aggfunc=pd.Series.nunique, margins=True).fillna(0))

#remove all na's
financials_df = financials_df.fillna(0).reset_index(drop=True)

In [ ]:
#remove all entries with missing valuations
#now we prepare the final dataset
#drop all accounts data that are sourced after 2016
financials_df = financials_df.loc[financials_df['Source_Year'] <2017].reset_index(drop=True)
#drop all valuation amounts other than the two we want to use
display(pd.pivot_table(financials_df, index=['Source_Year'], columns=[], values=['Symbol'], aggfunc=pd.Series.nunique, margins=True).fillna(0))

#sort the financials by stock symbol, year and month 
financials_df=financials_df.sort_values(['Symbol', 'Source_Year','Source_Month','Source_Day'], ascending=[True, False,False,False]).reset_index(drop=True)

display(financials_df)
#write to disk
financials_df.to_csv(model_data_path,sep='\t')

In [ ]:
#check the datatypes
for col in financials_df.columns:
    if (financials_df[col].dtype == 'O'):
        print(col)
        print(financials_df[col].unique())
        print('')
display(dict(zip(financials_df.columns,financials_df.dtypes)))